In [1]:
# from randtest import randtest
import numpy as np
import pandas
import pickle
from tqdm import tqdm
import random
import tensorflow.keras.backend as K

2024-08-05 13:31:15.497372: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 13:31:16.413068: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 13:31:28.900842: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
print(cores)

80


In [3]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [4]:
def forecast_loss(y_true, y_pred):
    V=134
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

In [5]:
def score(y_preds, y_true):
    y_pred = y_preds.reshape(-1, 134)
    mmse = forecast_loss(y_true, y_pred)
    s = 0
    for i in mmse:
        s += i
    return s/len(mmse)

In [6]:
def ra_test(a_0, b_0, y_true, r=1000):
    """randomization test"""
    if len(a_0) != len(b_0):
        print('Input A and B are not eaqul-sized!')
        return
    delta_s_rs = []
    c = 0
    n = len(a_0)
    
    
    delta_s_0 = score(a_0, y_true) - score(b_0, y_true)
    delta_s_0 = delta_s_0.numpy()
    
    for _ in tqdm(range(r)):
        a_r = []
        b_r = []
        # shuffle
        for i in range(n):
            # swap
            rn = random.randint(0, 1)
            # swap
            if rn == 0:
                a_r.append(b_0[i])
                b_r.append(a_0[i])
            # do not swap
            else:
                a_r.append(a_0[i])
                b_r.append(b_0[i])

        a_r = np.array(a_r)
        b_r = np.array(b_r)
        delta_s_r = score(a_r, y_true) - score(b_r, y_true)
        delta_s_r = delta_s_r.numpy()
        # print(delta_s_r)
        # print(abs(delta_s_r))
        delta_s_rs.append(delta_s_r)
        if abs(delta_s_r) >= abs(delta_s_0):
            c += 1
        
    print(c)
    p = c/r
    print(delta_s_rs)
    return p

In [7]:
baseline_path = 'randomization_test/data/baseline.pkl'
baseline_data, var_to_ind = pickle.load(open(baseline_path, 'rb'))

forecasting_preds = []

# y_true for score
y_true = []
for y in baseline_data['forecasting_test_op']:
    y_true.append(y)
y_true = np.array(y_true)

for pred in baseline_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)
x = forecasting_preds.flatten()
del baseline_data, forecasting_preds

In [8]:
m_path = 'randomization_test/data/hierar-early.pkl'
m_data, var_to_ind_m = pickle.load(open(m_path, 'rb'))
m_data

forecasting_preds = []

for pred in m_data['forecasting_pred']:
    forecasting_preds.append(np.array(pred))

forecasting_preds = np.array(forecasting_preds)

y = forecasting_preds.flatten()
del m_data, forecasting_preds

In [9]:
p = ra_test(x, y, y_true = y_true, r=100)
p

100%|██████████| 100/100 [1:43:53<00:00, 62.34s/it]

69
[-0.006627579801953054, -0.01649280698524258, -0.018199547325626675, 0.009925247432768458, 0.006374101174191793, -0.005155588038938674, 0.012375433192049812, 0.002749296843703064, 0.0005521676290669575, -0.0096353593577847, 0.0016290092902204734, 0.008332295362564857, 0.011101215166004152, -0.00853454555336608, 0.009277946827430661, 0.007623103736237979, -0.004893038495020541, 0.007615326400554068, 0.011523878229470874, 0.0027698115626062503, -0.005727494220489682, 0.0004987542843579718, -0.009937937200610314, 0.013800191459366395, -0.01257656116678696, -0.00685969363520833, -0.013586261654524812, -0.020527636950712136, -0.0055532848753605535, 0.002474630763698471, 0.001719507484740923, 0.010180312602921404, -0.013522470039117884, 0.008747091893087422, -0.012572594225525258, 0.010983535441337011, -0.008061477896392333, 0.007095044779497073, 0.010365560396094153, -0.008105801898126153, -0.002582586318715663, 0.006540134662210484, 0.007459650772720039, 0.00021450394608102386, -0.02336

0.69

In [11]:
p

0.69